#Homework 4.3

- Using the blog.py script to run our blog (and doing a mongoimport of the documents in the posts.json file that is provided), our assignment is to make the following blog pages fast:

    - The blog home page
    - The page that displays blog posts by tag (http://localhost:8082/tag/whatever)
    - The page that displays a blog entry by permalink (http://localhost:8082/post/permalink)

- We can make the pages fast by creating indexes for the MongoDB databases that get used by the blog-related code. So, let's try to homepage first.

# Speeding up the homepage

In [2]:
# Here's the relevant code from blog.py that starts the homepage:
# This route is the main page of the blog
#@bottle.route('/')
#def blog_index():
#
#    cookie = bottle.request.get_cookie("session")
#
#    username = sessions.get_username(cookie)
#
#    # even if there is no logged in user, we can show the blog
#    l = posts.get_posts(10)
#
#    return bottle.template('blog_template', dict(myposts=l, username=username))

In [3]:
# As you can see, there are two places that probably concern us:
#   1) username = sessions.get_username(cookie)
#   2) l = posts.get_posts(10)
# (the cookie = bottle.request.get_cookie("session") seems to be just
# some bottle framework-related code and doesn't concern MongoDB at all)

In [4]:
# A quick look at blogPostDAO.py and sessionDAO.py confirm that the
# get functions are of course looking in those respective collections

In [5]:
# Let's take a look at the sessions collection first
import pymongo
connection_string = 'mongodb://localhost:27017'
connection = pymongo.MongoClient(connection_string)
db = connection.blog

In [7]:
list(db.sessions.find().limit(1))

[{u'_id': u'YCcrWrkAtfUYPSLmnsRmULTPuTHsheeg', u'username': u'mulhod'}]

In [9]:
# In the sessionDAO.py code, the get_username function is called,
# which, in turn, calls the get_session function, which runs the
# following code:
#session = self.sessions.find_one({'_id': session_id})
# So, as you can, see it does a search on the '_id' keys
# However, we know that _id keys are indexed by default...
list(db.sessions.index_information())

[u'_id_']

In [10]:
# As you can see, the collection is indeed indexed on _id already,
# so there is little we can do to speed up this part

In [11]:
# However, let's take a look at the get_posts function and what it
# does

In [15]:
# get_posts function in blogPostDAO.py runs the following query:
#cursor = self.posts.find().sort('date', direction=-1).limit(num_posts)
# It wants to find some recent blog posts that it can put on the main
# page, so it looks for all documents and then sorts by the 'date' key
# and limits the number of returned docs.
# Does this collection have any indexes (other than _id: 1)?
list(db.posts.index_information())

[u'_id_']

In [16]:
# It appears not.
# So, an obvious thing we can do here is make an index on the 'date' key,
# sorting in reverse since that is exactly what this code wants (we don't
# really need to do it in reverse, actually, but let's do it anyway)

In [30]:
# Actually, let's first get the timing information for comparison
%timeit db.posts.find().sort('date', direction=-1)

The slowest run took 5.36 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 15.8 µs per loop


In [31]:
# Now, we create the index and then rerun the query
db.posts.create_index([('date', pymongo.DESCENDING)])

u'date_-1'

In [33]:
%timeit db.posts.find().sort('date', direction=-1)

The slowest run took 4.75 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 15.4 µs per loop


In [34]:
# Well, the differences in execution time (crude, I know!) are
# probably not very significant, but maybe this index sped it up a
# tad bit

#Speeding up the posts by-tag page

In [35]:
# The posts_by_tag function, which produces this page, also gets
# some info from the sessions collection, but, as we've seen,
# there's nothing we can do to speed up the retrieval of _id keys

In [37]:
# Besides that, the code also calls the get_username function,
# which we have already tried to speed up.
# The only other call it makes is to the get_posts_by_tag method of
# the blogPostDAO.py module and here is the relevant line of code:
#
#cursor = self.posts.find({'tags':tag}).sort('date', direction=-1).limit(num_posts)
#

In [38]:
# So, it looks like we should make an index that uses both the tag
# and the date keys

In [40]:
# Let's first see how long it would take to run a query like that
# above without any extra indexes added in.

In [42]:
# Here's an example doc
list(db.posts.find())[0]

{u'_id': ObjectId('50ab0f8bbcf1bfe2536dc3f9'),
 u'author': u'machine',
 u'body': u'Amendment I\n<p>Congress shall make no law respecting an establishment of religion, or prohibiting the free exercise thereof; or abridging the freedom of speech, or of the press; or the right of the people peaceably to assemble, and to petition the Government for a redress of grievances.\n<p>\nAmendment II\n<p>\nA well regulated Militia, being necessary to the security of a free State, the right of the people to keep and bear Arms, shall not be infringed.\n<p>\nAmendment III\n<p>\nNo Soldier shall, in time of peace be quartered in any house, without the consent of the Owner, nor in time of war, but in a manner to be prescribed by law.\n<p>\nAmendment IV\n<p>\nThe right of the people to be secure in their persons, houses, papers, and effects, against unreasonable searches and seizures, shall not be violated, and no Warrants shall issue, but upon probable cause, supported by Oath or affirmation, and partic

In [44]:
# Let's search for posts with the tag 'watchmaker'
%timeit db.posts.find({'tags': 'watchmaker'}).sort('date', direction=-1)

The slowest run took 13.15 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 14.6 µs per loop


In [45]:
# Now let's add a multi-key index with both tag and date and
# then we can rerun the query
# Note: Let's make both keys be descending since the most recent posts
# are what's wanted and since it really doesn't matter that much
# whether the tags keys are ascending or descending
db.posts.create_index([('tags', pymongo.DESCENDING),
                       ('date', pymongo.DESCENDING)])

u'tags_-1_date_-1'

In [49]:
%timeit db.posts.find({'tags': 'watchmaker'}).sort('date', direction=-1)

The slowest run took 4.91 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 15.9 µs per loop


In [52]:
# That doesn't really seem to have done the trick...
# Let's remove the index and maybe try something else
db.posts.drop_index('tags_-1_date_-1')

In [53]:
# Let's just try to make a 'tags' index, then
db.posts.create_index('tags')

u'tags_1'

In [61]:
%timeit db.posts.find({'tags': 'watchmaker'}).sort('date', direction=-1)

The slowest run took 5.93 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 15.8 µs per loop


In [62]:
# According to output from the explain() method in the mongo shell,
# I was able to determine that both the indexes I have tried here
# would be useful and would actually win out over the _id index or the
# date: -1 index that was created for the first part of this homework.
# The date: -1 index is tried and rejected and the _id index, if used,
# results in total document lookup of 1000 or more (about one hundred
# times larger than when it uses the tags or tags: -1 + date: -1
# indexes.
# The execution stats for the two indexes tried here seem to be
# identical.
# Thus, all things being equal (and chalking up the seeming %timeit
# disagreement to pymongo overhead and the queries taking close to zero
# time, relatively speaking), I believe it best to stick with the
# tags index.